In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'/Users/apple/Documents/Personal/Computer Vision and Deep Learning/pc-parts'

In [4]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/hbbolaji/pc-parts-classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "hbbolaji"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "4bdb5a1658312cbc56f3d871f4a979d11754c578"

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
  root_dir: Path
  model_path: Path
  all_params: dict
  mlflow_uri: str

In [6]:
# Configuration Manage
from pcpartsclassifier.constants import PARAMS_FILE_PATH, CONFIG_FILE_PATH
from pcpartsclassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
  def __init__(self,
               config = CONFIG_FILE_PATH,
               params = PARAMS_FILE_PATH) -> None:
    self.config = read_yaml(config)
    self.params = read_yaml(params)

    create_directories([self.config.artifacts_root])
  
  def get_evaluation_config(self) -> EvaluationConfig:
    config = self.config.evaluation
    evaluation_config = EvaluationConfig(
      root_dir = config.root_dir,
      model_path = config.model_path,
      all_params = self.params,
      mlflow_uri = config.mlflow_uri
    )
    return evaluation_config

In [7]:
# Evaluation Component
import torch
from pcpartsclassifier.config.configuration import ConfigurationManager as CM
from pcpartsclassifier.components.base_model import BaseModel
from pcpartsclassifier.components.model_trainer import Training
from pcpartsclassifier.utils.common import save_json
import mlflow
from urllib.parse import urlparse


class ModelEvaluation:
  def __init__(self, config: EvaluationConfig) -> None:
    self.config = config

    self.config_manager = CM()
  
  def get_eval_data(self):
    training_config = self.config_manager.get_training_config()
    training = Training(config=training_config)
    training.data_preparation()
    self.accuracy = training.accuracy
    self.validation_loader = training.validation_loader
    
  @staticmethod
  def load_model(model_path: Path, model_config):
    model = BaseModel(config=model_config)
    model.load_state_dict(torch.load(model_path))
    return model
  
  def evaluate(self):
    model_config = self.config_manager.get_base_model_config()
    criterion = torch.nn.CrossEntropyLoss()
    self.model = self.load_model(model_path=self.config.model_path,
                                 model_config=model_config)
    self.get_eval_data()
    self.model.eval()
    with torch.inference_mode():
      eval_loss, eval_acc = 0, 0
      for i, (images, labels) in enumerate(self.validation_loader):
        logits = self.model(images)
        loss = criterion(logits, labels)
        acc = self.accuracy(logits, labels)

        eval_loss += loss.item()
        eval_acc += acc.item()

      eval_loss /= len(self.validation_loader)
      eval_acc /= len(self.validation_loader)

      self.scores = eval_loss, 100 * eval_acc
  
  def save_score(self):
    scores = {'Loss': self.scores[0], 'Accuracy': self.scores[1]}
    save_json(path=Path('scores.json'), data=scores)

  def log_into_mlflow(self):
    tracking_uri = self.config.mlflow_uri
    mlflow.set_registry_uri(tracking_uri)

    tracking_uri_type_store = urlparse(mlflow.get_registry_uri()).scheme

    with mlflow.start_run():
      mlflow.log_params(self.config.all_params)
      mlflow.log_metrics({'loss': self.scores[0], 'accuracy': self.scores[1]})

      # model registery is not tracked with file store
      if tracking_uri_type_store != "file":
        mlflow.pytorch.log_model(self.model, 'model', registered_model_name="efficientnet_b0")
      else:
        mlflow.pytorch.log_model(self.model, 'model')

[2024-03-31 09:39:40,661 : INFO : utils : NumExpr defaulting to 4 threads.]


In [8]:
# pipeline
try:
  config = ConfigurationManager()
  evaluation_config = config.get_evaluation_config()
  evaluation = ModelEvaluation(config=evaluation_config)
  evaluation.evaluate()
  evaluation.save_score()
  evaluation.log_into_mlflow()
except Exception as e:
  raise e

[2024-03-31 09:39:41,631 : INFO : common : yaml file: config/config.yaml loaded successfull]
[2024-03-31 09:39:41,640 : INFO : common : yaml file: params.yaml loaded successfull]
[2024-03-31 09:39:41,651 : INFO : common : created directory at artifacts]
[2024-03-31 09:39:41,656 : INFO : common : yaml file: config/config.yaml loaded successfull]
[2024-03-31 09:39:41,664 : INFO : common : yaml file: params.yaml loaded successfull]
[2024-03-31 09:39:41,672 : INFO : common : created directory at artifacts]
[2024-03-31 09:39:41,869 : INFO : common : yaml file: config/config.yaml loaded successfull]
[2024-03-31 09:39:41,871 : INFO : common : yaml file: params.yaml loaded successfull]
[2024-03-31 09:39:41,872 : INFO : common : created directory at artifacts]
[2024-03-31 09:40:01,348 : INFO : common : json file saved at scores.json]
[2024-03-31 09:40:03,235 : INFO : file_utils : PyTorch version 2.0.1 available.]
[2024-03-31 09:40:07,558 : INFO : modeling_xlnet : Better speed can be achieved wi

2024/03/31 09:40:20 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2024-02-29; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'pcpartsclassifier'}
/Users/apple/anaconda3/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'efficientnet_b0' already exists. Creating a new version of this model...
2024/03/31 09:40:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: efficientnet_b0, version 2
Created version '2' of model 'efficientnet_b0'.
